<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/pivot-point-driven-pendulum-henry8675309/blob/main/Stability2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Pivot-Driven Pendulum Workflow


## 1. Physics Derivation (EOM)

**Goal:** Derive the Equation of Motion (EOM) for $\ddot{\theta}$ using the Lagrangian method ($L = T - V$).

**Coordinates:**
* **Pivot:**
    $$x_p(t) = -l \sin(\omega_d t)$$
    $$y_p(t) = l (\cos(\omega_d t) - 1)$$
* **Bob:**
    $$x_b(t) = x_p(t) + l \sin(\theta)$$
    $$y_b(t) = y_p(t) - l \cos(\theta)$$

**Velocities:**
$$\dot{x}_b = \dot{x}_p + l \dot{\theta} \cos(\theta)$$
$$\dot{y}_b = \dot{y}_p + l \dot{\theta} \sin(\theta)$$

**Lagrangian:**
$$T = \frac{1}{2}m(\dot{x}_b^2 + \dot{y}_b^2)$$
$$V = mgy_b$$

**Solve:**
$$\frac{d}{dt}\left(\frac{\partial L}{\partial \dot{\theta}}\right) - \frac{\partial L}{\partial \theta} = 0 \implies \ddot{\theta} = f(t, \theta, \dot{\theta}, \omega_d)$$

## 2. Code Implementation

* Import `numpy`, `matplotlib.pyplot`, `solve_ivp`, `FuncAnimation`, and `HTML`.
* Define `pendulum_ode(t, y, ...)`: Implements the EOM for $\ddot{\theta}$ from Part 1.
* Define `get_pivot_xy(t, ...)`: Implements the corrected pivot equations from the homework image.
* Define `get_pendulum_xy(theta, l)`: Calculates bob's position relative to pivot.
* Define `get_lab_xy(...)`: Calculates bob's position in the lab frame.
* Copy `create_pendulum_animation(...)` from the `README.md` file.

## 3. Simulation & Visualization

* **1. Set Parameters:** `l`, `g`, `omega_d`.
* **2. Set Initial Conditions:** $\theta_0=0$ and $\dot{\theta}_0=0$.
* **3. Set Time:** `t_plot = np.linspace(0, 30, 1000)`.
* **4. Solve ODE:** `sol = solve_ivp(pendulum_ode, ...)`.
* **5. Plot Angle vs. Time:** `plt.plot(sol.t, sol.y[0])`. This is our main stability check.
* **6. Animate:** Use helper functions and `create_pendulum_animation` to create the video.

## 4. Stability Analysis

* **1. Analytic (Text Cell):** Linearize the EOM from Part 1 for "nearly stationary" motion using small-angle approximations:
    $$\sin(\theta) \approx \theta$$
    $$\cos(\theta) \approx 1$$
    Analyze the resulting linear equation to predict at which frequencies $\omega_d$ parametric resonance (instability) occurs, relative to $\omega_0 = \sqrt{g/l}$.
* **2. Computational (Code Cell):** Test the analytic predictions by running the simulation (Part 3) with different `omega_d` values (e.g., `0.5*omega_0`, `2.0*omega_0`).

## 5. Conclusion (Text Cell)

* Write a final summary comparing the analytic predictions for instability with the results from the "Angle vs. Time" plots.




In [4]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [7]:
# --- Define global parameters ---
g = 9.81  # acceleration due to gravity
l = 1.0   # pendulum length
omega_d = 2.0  # Driving frequency (you will change this to test stability)

# --- Define the Equation of Motion ---
def pendulum_ode(t, y, l, g, omega_d):
    """
    The EOM for the pivot-driven pendulum.
    """
    theta, dtheta = y # y[0] is theta, y[1] is dtheta

    # d2theta is the EOM you derived in Part 2.
    # It will be a function of t, theta, dtheta, l, g, and omega_d
    # Example (THIS IS NOT THE ANSWER, just a placeholder):
    # d2theta = -(g/l) * np.sin(theta) + np.cos(omega_d * t) * np.sin(theta)

    d2theta = ... # <-- REPLACE THIS with your derived equation

    return [dtheta, d2theta]

# --- Define Coordinate Helper Functions ---
def get_pivot_xy(t, l, omega_d):
    """
    Calculates pivot coordinates based on the CORRECTED parameterization.
    """
    x_p = -l * np.sin(omega_d * t)
    y_p = l * (np.cos(omega_d * t) - 1)
    return x_p, y_p

def get_pendulum_xy(theta, l):
    """
    Calculates bob coordinates relative to the pivot.
    """
    x_pendulum = l * np.sin(theta)
    y_pendulum = -l * np.cos(theta)
    return x_pendulum, y_pendulum

def get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum):
    """
    Calculates bob coordinates in the lab frame.
    """
    x_lab = x_pivot + x_pendulum
    y_lab = y_pivot + y_pendulum
    return x_lab, y_lab

In [2]:
# create pendulum animation
import matplotlib.pyplot as plt # Import matplotlib.pyplot

def create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=1, show_traj=False, speed_factor=1.0):
    """
    Creates an animation of the pivot-driven pendulum.

    PARAMETERS:
    t : ndarray
        Time array.
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    x_lab : ndarray
        x-positions of the pendulum bob in the lab frame over time.
    y_lab : ndarray
        y-positions of the pendulum bob in the lab frame over time.
    l : float, optional
        pendulum length (default is 1).
    show_traj : bool, optional
        toggle showing the trajectory of the pendulum bob (default is False).
    speed_factor : float, optional
        factor to scale the animation speed (default is 1.0).
        a value > 1.0 slows down the animation, < 1.0 speeds it up.

    RETURNS:
    anim : FuncAnimation
        matplotlib animation object.
    """
    fig, ax = plt.subplots(figsize=(4, 4)) # Increased figure size
    ax.set_xlim(np.min(x_pivot) - l, np.max(x_pivot) + l)
    ax.set_ylim(np.min(y_pivot) - l, np.max(y_pivot) + l)
    # ax.set_aspect('equal', adjustable='box')
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    # Initialize plot elements
    pivot, = ax.plot([1], [1], 'o', color='black', markersize=8, label='Pivot')
    pendulum_arm, = ax.plot([0], [1], '-', color='black', lw=2, label='Pendulum Arm')
    pendulum_bob, = ax.plot([1], [3], 'o', color='red', markersize=12, label='Pendulum Bob')
    trajectory = None # Initialize trajectory to None

    if show_traj:
        trajectory, = ax.plot([1], [1], '-', color='green', lw=1, alpha=0.5, label='Trajectory') # Add trajectory line

    def animate(i):
        # Update the positions of the plot elements
        pivot.set_data([x_pivot[i]], [y_pivot[i]]) # Pass as sequences
        pendulum_arm.set_data([x_pivot[i], x_lab[i]], [y_pivot[i], y_lab[i]])
        pendulum_bob.set_data([x_lab[i]], [y_lab[i]]) # Pass as sequences

        artists = [pivot, pendulum_arm, pendulum_bob] # List of artists to update

        if show_traj and trajectory:
             trajectory.set_data(x_lab[:i+1], y_lab[:i+1]) # Update trajectory data
             artists.append(trajectory) # Add trajectory to the list of artists

        return artists # Return all updated artists

    # Create the animation
    # Adjust the interval based on the average time step in t
    # This aims to make the animation speed consistent with the simulation time
    average_time_step = np.mean(np.diff(t))
    # Scale the interval by the speed_factor
    interval = average_time_step * 1000 * speed_factor # Convert to milliseconds and apply speed_factor.

    anim = FuncAnimation(fig, animate, frames=len(t), interval=interval, blit=True)
    plt.close(fig) # Close the initial figure to prevent it from displaying

    return anim

In [ ]:
# animate motion
# plot pendulum angle versus time
# get coordinates for plotting
import numpy as np # Import numpy library
# Define t_min and t_max
t_min = 0 # start time
t_max = 10 # end time
n_points = 300 # number of frames
t_plot = np.linspace(t_min, t_max, n_points)
theta_plot = sol.sol(t_plot)[0] # requires `dense_output=True` in `solve_ivp`
x_pivot, y_pivot = get_pivot_xy(t_plot, omega_d, x_p0, y_p0) # pivot coordinates
x_pendulum, y_pendulum = get_pendulum_xy(t_plot, theta_plot, l) # pendulum coordinates of pendulum (referenced to pivot point)
x_lab, y_lab = get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum) # pendulum coordinates in the lab frame

# animation flags/parameters
show_traj = True # True --> show trajectory; False --> do not show trajectory
speed_factor = 5 # >1 --> slow down animation; <1 --> speed up animation

# create_pendulum_animation
pendulum_animation = create_pendulum_animation(t_plot, x_pivot, y_pivot, x_lab, y_lab, show_traj=show_traj, speed_factor=speed_factor) # create animation object with trajectory shown
HTML(pendulum_animation.to_html5_video()) # display animation

In [1]:
# Plot the pendulum angle versus time
import matplotlib.pyplot as plt # Import matplotlib.pyplot
plt.figure(figsize=(8, 4))
plt.plot(t_plot, theta_plot)
plt.xlabel('Time (s)')
plt.ylabel('Pendulum Angle (radians)')
plt.title('Pendulum Angle vs. Time')
plt.grid(True)
plt.show()

NameError: name 't_plot' is not defined

<Figure size 800x400 with 0 Axes>

In [8]:
# define functions to get coordinates for plotting
import numpy as np # Import numpy library

def get_pivot_xy(t, omega_d, x_p0, y_p0):
    """
    Calculates the x and y coordinates of the pivot point over time.

    PARAMETERS:
    t : ndarray
        Time array.
    omega_d : float
        Driving frequency.
    x_p0 : float
        Initial x-position of the pivot.
    y_p0 : float
        Initial y-position of the pivot.

    RETURNS:
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    """
    # Example: Oscillating pivot in x-direction
    x_pivot = x_p0 + 0.5 * np.sin(omega_d * t)  # Example: Add an oscillation amplitude of 0.5
    y_pivot = y_p0 + 0.1 * t  # Added linear motion in the y-direction over time
    return x_pivot, y_pivot

def get_pendulum_xy(t, theta, l):
    """
    Calculates the x and y coordinates of the pendulum bob relative to the pivot.

    PARAMETERS:
    t : ndarray
        Time array.
    theta : ndarray
        Pendulum angle over time (in radians).
    l : float
        Pendulum length.

    RETURNS:
    x_pendulum : ndarray
        x-positions of the pendulum bob relative to the pivot over time.
    y_pendulum : ndarray
        y-positions of the pendulum bob relative to the pivot over time.
    """
    x_pendulum = l * np.sin(theta)
    y_pendulum = -l * np.cos(theta) # Negative because pendulum hangs down
    return x_pendulum, y_pendulum

def get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum):
    """
    Calculates the x and y coordinates of the pendulum bob in the lab frame.

    PARAMETERS:
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    x_pendulum : ndarray
        x-positions of the pendulum bob relative to the pivot over time.
    y_pendulum : ndarray
        y-positions of the pendulum bob relative to the pivot over time.
    """
    x_lab = x_pivot + x_pendulum
    y_lab = y_pivot + y_pendulum
    return x_lab, y_lab

# Define initial pivot position
x_p0 = 0.0
y_p0 = 0.0

In [3]:
from scipy.integrate import solve_ivp

# Define the differential equation for the pendulum (replace with your actual equation)
def pendulum_ode(t, y, l): # Removed omega_d, gamma, and A from parameters
    theta, dtheta = y
    # Equation for a simple pendulum (no driving or damping)
    dydt = [dtheta,
            -(9.81/l) * np.sin(theta)] # Removed damping and driving terms
    return dydt

# Define initial conditions and parameters (replace with your actual values)
theta0 = np.pi / 2 # initial angle (starting from horizontal for a full circle)
dtheta0 = 3 # initial angular velocity (needs to be large enough for a full circle)
l = 1 # pendulum length
# Removed omega_d, gamma, and A as they are not used in the simple pendulum equation

# Solve the differential equation
t_span = [t_min, t_max] # time span for integration
y0 = [theta0, dtheta0] # initial conditions
sol = solve_ivp(pendulum_ode, t_span, y0, dense_output=True, args=(l,)) # solve with dense output and pass pendulum length

NameError: name 'np' is not defined